In [1]:
import numpy as np

#状态转移概率矩阵
#很显然,状态4(第5行)就是重点了,要进入状态4,只能从状态2,3进入
#[5, 5]
P = np.array([
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 0.0],
])

#反馈矩阵，-100的位置是不可能走到的
#[5, 5]
R = np.array([
    [-1.0, 0.0, -100.0, -100.0, -100.0],
    [-1.0, -100.0, -2.0, -100.0, -100.0],
    [-100.0, -100.0, -100.0, -2.0, 0.0],
    [-100.0, 1.0, 1.0, 1.0, 10.0],
    [-100.0, -100.0, -100.0, -100.0, -100.0],
])

P, R

(array([[0.5, 0.5, 0. , 0. , 0. ],
        [0.5, 0. , 0.5, 0. , 0. ],
        [0. , 0. , 0. , 0.5, 0.5],
        [0. , 0.1, 0.2, 0.2, 0.5],
        [0. , 0. , 0. , 0. , 0. ]]),
 array([[  -1.,    0., -100., -100., -100.],
        [  -1., -100.,   -2., -100., -100.],
        [-100., -100., -100.,   -2.,    0.],
        [-100.,    1.,    1.,    1.,   10.],
        [-100., -100., -100., -100., -100.]]))

In [2]:
import numpy as np
import random


#生成一个chain
def get_chain(max_lens):
    #采样结果
    ss = []
    rs = []

    #随机选择一个除4以外的状态作为起点
    s = random.choice(range(4))
    ss.append(s)

    for _ in range(max_lens):
        #按照P的概率，找到下一个状态
        s_next = np.random.choice(np.arange(5), p=P[s])

        #取到r
        r = R[s, s_next]

        #s_next变成当前状态,开始接下来的循环
        s = s_next

        ss.append(s)
        rs.append(r)

        #如果状态到了4则结束
        if s == 4:
            break

    return ss, rs


get_chain(20)

([2, 3, 4], [-2.0, 10.0])

In [3]:
#生成N个chain
def get_chains(N, max_lens):
    ss = []
    rs = []
    for _ in range(N):
        s, r = get_chain(max_lens)
        ss.append(s)
        rs.append(r)

    return ss, rs


ss, rs = get_chains(100, 20)

ss, rs

([[2, 4],
  [0, 0, 0, 1, 0, 1, 2, 3, 4],
  [0, 1, 0, 1, 0, 0, 1, 2, 3, 4],
  [1, 2, 4],
  [3, 4],
  [1, 2, 4],
  [1, 0, 1, 0, 0, 1, 2, 3, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 4],
  [2, 4],
  [2, 3, 4],
  [2, 4],
  [0, 1, 0, 0, 0, 1, 2, 4],
  [3, 4],
  [1, 2, 3, 1, 0, 1, 0, 1, 2, 4],
  [2, 3, 3, 3, 2, 4],
  [1, 2, 4],
  [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 4],
  [0, 1, 2, 4],
  [2, 4],
  [2, 4],
  [2, 4],
  [2, 4],
  [1, 2, 4],
  [2, 4],
  [3, 2, 3, 1, 2, 3, 4],
  [0, 0, 1, 2, 3, 3, 3, 4],
  [2, 3, 2, 4],
  [2, 4],
  [2, 4],
  [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
  [1, 2, 4],
  [2, 3, 3, 4],
  [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 3, 4],
  [1, 2, 3, 4],
  [0, 1, 2, 3, 4],
  [1, 0, 0, 0, 0, 1, 0, 1, 2, 3, 1, 2, 4],
  [2, 3, 4],
  [2, 4],
  [2, 4],
  [3, 3, 1, 2, 3, 2, 3, 4],
  [3, 4],
  [0, 1, 2, 3, 3, 4],
  [3, 4],
  [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 2, 3, 4],
  [3, 3, 4],
  [2, 3, 4],
  [1, 0, 1, 0, 1, 2, 3, 4],
  [0, 1, 2, 3, 3, 3, 4],
  [1, 0, 0, 0, 1, 0,

In [4]:
#给定一条链,计算回报
def get_value(rs):
    sum = 0
    for i, r in enumerate(rs):
        #给每一步的反馈做一个系数,随着步数往后衰减,也就是说,越早的动作影响越大
        sum += 0.5**i * r

    #最终的反馈是所有步数衰减后的求和
    return sum


get_value(rs[0])

0.0

In [5]:
#蒙特卡洛法评估每个状态的价值
def get_values_by_monte_carlo(ss, rs):
    #记录5个不同开头的价值
    #其实只有4个,因为状态4是不可能作为开头状态的
    values = [[] for _ in range(5)]

    #遍历所有链
    for s, r in zip(ss, rs):
        #计算不同开头的价值
        values[s[0]].append(get_value(r))

    #求每个开头的平均价值
    return [np.mean(i) for i in values]


#-1.228923788722258,-1.6955696284402704,0.4823809701532294,5.967514743019431,0
get_values_by_monte_carlo(*get_chains(2000, 20))

/root/anaconda3/envs/cpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/root/anaconda3/envs/cpu/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[-1.2689316385800076,
 -1.595738185128587,
 0.5337043907456025,
 5.91608556615244,
 nan]

In [6]:
#计算状态动作对(s,a)出现的频率,以此来估算策略的占用度量
def occupancy(ss, rs, s, a):
    rho = 0

    count_by_time = np.zeros(max_time)
    count_by_s_a = np.zeros(max_time)

    for si, ri in zip(ss, rs):
        for i in range(len(ri)):
            s_opt = si[i]
            a_opt = si[i + 1]

            #统计每个时间步的次数
            count_by_time[i] += 1

            #统计s，a出现的次数
            if s == s_opt and a == a_opt:
                count_by_s_a[i] += 1

    #i -> [999 - 0]
    for i in reversed(range(max_time)):
        if count_by_time[i] == 0:
            continue

        #以时间逐渐衰减
        rho += 0.5**i * count_by_s_a[i] / count_by_time[i]

    return (1 - 0.5) * rho


max_time = 1000
ss, rs = get_chains(max_time, 2000)

#0.112567796310472
occupancy(ss, rs, 3, 1) + occupancy(ss, rs, 3, 2) + occupancy(ss, rs, 3, 3)

0.11304324114416356

In [7]:
#重新定义状态转移概率矩阵
P = np.array([
    [0.6, 0.4, 0.0, 0.0, 0.0],
    [0.3, 0.0, 0.7, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.18, 0.36, 0.36, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0],
])

ss, rs = get_chains(max_time, 2000)

#0.23199480615618912
occupancy(ss, rs, 3, 1) + occupancy(ss, rs, 3, 2) + occupancy(ss, rs, 3, 3)

0.23167624185977734